In [3]:
import numpy as np
import json
import pandas as pd

### 配置来源

使用  precompute\data\1\1001 目录下的json文件作为数据源；然后整理出需要的numpy数组，以加速并行计算；
举例：
<!-- TODO: agent
这一部分准备在ipynb中一步步摸索，然后把案例整理到这里
-->
* 下面我们一步步的找数据，进行探索和整合：
* 对于 D:\fishinggame\ExportedData\Fishing_1006001_Dense_20260107_154037\Fishing_1006001_Global.npy，
先截取中其中的1006001，也就是scene id，然后到D:\fishinggame\precompute\data\1\1001\map_scene.json中，找到对应的map id

In [2]:
import re
from pathlib import Path

# 配置路径
DATA_ROOT = Path(r'D:\fishinggame\precompute\data\1\1001')
EXPORTED_DATA_ROOT = Path(r'D:\fishinggame\ExportedData')

# 加载 map_scene.json
with open(DATA_ROOT / 'map_scene.json', 'r', encoding='utf-8') as f:
    map_scene = json.load(f)

# 建立 assetId -> map_id 的反向索引
asset_to_map = {info['assetId']: int(map_id) for map_id, info in map_scene.items() if info.get('assetId')}
print(f'已加载 {len(map_scene)} 个地图配置，其中 {len(asset_to_map)} 个有 assetId')
print(f'assetId -> map_id 映射: {asset_to_map}')

已加载 9 个地图配置，其中 6 个有 assetId
assetId -> map_id 映射: {'1001001': 1001, '1002001': 1002, '1003001': 1003, '1004001': 1004, '1004002': 1008, '1006001': 1009}


In [3]:
def get_scene_id_from_path(npy_path: str) -> str:
    """从npy文件路径中提取scene_id (如 Fishing_1006001_Global.npy -> '1006001')"""
    match = re.search(r'Fishing_(\d+)', str(npy_path))
    if not match:
        raise ValueError(f'无法从路径中提取scene_id: {npy_path}')
    return match.group(1)

def get_map_id_from_scene_id(scene_id: str) -> int:
    """根据scene_id查找对应的map_id"""
    if scene_id in asset_to_map:
        return asset_to_map[scene_id]
    raise ValueError(f'找不到scene_id {scene_id} 对应的map_id')

def get_map_id_from_npy_path(npy_path: str) -> int:
    """从npy文件路径直接获取map_id"""
    scene_id = get_scene_id_from_path(npy_path)
    return get_map_id_from_scene_id(scene_id)

# 测试示例
test_path = r'D:\fishinggame\ExportedData\Fishing_1006001_Dense_20260107_154037\Fishing_1006001_Global.npy'
scene_id = get_scene_id_from_path(test_path)
map_id = get_map_id_from_scene_id(scene_id)
print(f'文件路径: {test_path}')
print(f'提取的 scene_id: {scene_id}')
print(f'对应的 map_id: {map_id}')
print(f'地图信息: {map_scene[str(map_id)]}')

文件路径: D:\fishinggame\ExportedData\Fishing_1006001_Dense_20260107_154037\Fishing_1006001_Global.npy
提取的 scene_id: 1006001
对应的 map_id: 1009
地图信息: {'id': 1009, 'name': 'map_base_6', 'desc': 106, 'assetId': '1006001', 'originOffsetX': 0, 'originOffsetY': 0, 'offsetX': 0, 'offsetY': 0, 'sizeX': 1000, 'sizeY': 1000, 'rotate': 0, 'mark': ''}


#### 对于局部池取stock
* 
* 去D:\fishinggame\precompute\data\1\1001\fish_stock.json当中，

In [4]:
# 加载额外的配置表
with open(DATA_ROOT / 'fish_pond_list.json', 'r', encoding='utf-8') as f:
    fish_pond_list = json.load(f)

with open(DATA_ROOT / 'fish_stock.json', 'r', encoding='utf-8') as f:
    fish_stock_config = json.load(f)

print(f"已加载 {len(fish_pond_list)} 个鱼塘配置")
print(f"已加载 {len(fish_stock_config)} 个 Stock 配置")

# 获取 map_id 对应的 map_scene 配置
current_map_info = map_scene.get(str(map_id))
if not current_map_info:
    raise ValueError(f"Found no map info for id {map_id}")

map_desc_id = current_map_info.get('desc')
print(f"\n当前地图: {current_map_info['name']} (ID: {map_id})")
print(f"关联的 Desc ID (用于对应 fish_pond_list.mapId): {map_desc_id}")

# 查找关联的 Pond 和 Stock
print(f"\n查找 mapId == {map_desc_id} 的鱼塘...")
related_ponds = [pond for pond in fish_pond_list.values() if pond.get('mapId') == map_desc_id]

if not related_ponds:
    print("警告: 未找到关联的鱼塘配置 (Pond)")
else:
    print(f"找到 {len(related_ponds)} 个关联鱼塘:")
    for pond in related_ponds:
        # 兼容处理: 有些json key可能是str类型的id
        pond_id = pond.get('id')
        pond_name = pond.get('name')
        stock_id = pond.get('fishStockId')
        
        print(f"  - Pond: {pond_name} (ID: {pond_id}) -> Stock ID: {stock_id}")
        
        # 查询 Stock 详情 (注意 key 可能是字符串)
        stock_info = fish_stock_config.get(str(stock_id))
        if stock_info:
             print(f"    Stock 详情: Name={stock_info.get('name')}, ResetTime={stock_info.get('resetDayTime')}")
        else:
             print(f"    警告: 在 fish_stock.json 中未找到 Stock ID {stock_id}")

已加载 7 个鱼塘配置
已加载 6 个 Stock 配置

当前地图: map_base_6 (ID: 1009)
关联的 Desc ID (用于对应 fish_pond_list.mapId): 106

查找 mapId == 106 的鱼塘...
找到 1 个关联鱼塘:
  - Pond: Sunset_Stream (ID: 301020005) -> Stock ID: 301030106
    Stock 详情: Name=stock_sunset, ResetTime=05:00


In [1]:
from pathlib import Path
import json

# Define the correct data root for this task (2004 directory)
DATA_ROOT_2004 = Path(r'D:\fishinggame\precompute\data\1\2004')

def load_json(filename):
    with open(DATA_ROOT_2004 / filename, 'r', encoding='utf-8') as f:
        return json.load(f)

print("Loading 2004 configuration files...")
stock_release_conf = load_json('stock_release.json')
fish_release_conf = load_json('fish_release.json')
basic_fish_quality_conf = load_json('basic_fish_quality.json')
basic_fish_species_conf = load_json('basic_fish_species.json')
fish_env_affinity_conf = load_json('fish_env_affinity.json')
fish_stock_conf_2004 = load_json('fish_stock.json')

print("Configuration files loaded successfully.")

Loading 2004 configuration files...
Configuration files loaded successfully.


In [4]:
# Assemble Precompute Data

all_assembled_items = []

# Iterate over stock_release entries
for config_id, entry in stock_release_conf.items():
    stock_id = entry.get('stockId')
    release_id = entry.get('releaseId')
    fish_id = entry.get('fishId')
    fish_env_id = entry.get('fishEnvId')
    
    # 1. Get Release Config
    release_conf = fish_release_conf.get(str(release_id))
        
    # 2. Get Species Info
    # fishId maps to 'species' in basic_fish_quality
    quality_conf = basic_fish_quality_conf.get(str(fish_id))
    species_id = None
    species_conf = None
    if quality_conf:
        species_id = quality_conf.get('species')
        if species_id:
             species_conf = basic_fish_species_conf.get(str(species_id))
    
    # 3. Get Env Affinity
    env_conf = fish_env_affinity_conf.get(str(fish_env_id))
    
    # 4. Construct the assembled object (Flattened for DataFrame)
    assembled_item = {
        'id': entry.get('id'), # The ID from stock_release (e.g. 4000010)
        'stockId': stock_id,
        'releaseId': release_id,
        'fishId': fish_id,
        'fishEnvId': fish_env_id,
        'speciesId': species_id,
        
        # Flatten key properties for easy access in DataFrame
        'fishName': quality_conf['name'] if quality_conf else None,
        'speciesName': species_conf['showName'] if species_conf else None,
        'envName': env_conf['name'] if env_conf else None,
        
        # Store full configs as dicts (or you could flatten further if needed)
        'releaseConfig': release_conf,
        'qualityConfig': quality_conf,
        'speciesConfig': species_conf,
        'envConfig': env_conf
    }
    
    all_assembled_items.append(assembled_item)

# Create Pandas DataFrame
stockFishesConfNp = pd.DataFrame(all_assembled_items)

print(f"Assembled DataFrame with {len(stockFishesConfNp)} rows.")
print("DataFrame Columns:", stockFishesConfNp.columns.tolist())

# Verify with a sample
sample_stock_id = 301030102
print(f"\nSample Data for Stock {sample_stock_id} (head 5):")
sample_df = stockFishesConfNp[stockFishesConfNp['stockId'] == sample_stock_id]

if not sample_df.empty:
    # Display first few rows with selected columns
    display_cols = ['id', 'releaseId', 'fishId', 'fishName', 'speciesName', 'envName']
    print(sample_df[display_cols].head().to_string())
else:
    print(f"Sample stock {sample_stock_id} not found.")

Assembled DataFrame with 414 rows.
DataFrame Columns: ['id', 'stockId', 'releaseId', 'fishId', 'fishEnvId', 'speciesId', 'fishName', 'speciesName', 'envName', 'releaseConfig', 'qualityConfig', 'speciesConfig', 'envConfig']

Sample Data for Stock 301030102 (head 5):
        id  releaseId     fishId        fishName speciesName              envName
0  4000010     300010  101031022   Alewife_Young         灰西鲱   Fish_Alewife_Young
1  4000020     300020  101031004  Alewife_Common         灰西鲱  Fish_Alewife_Common
2  4000030     300030  101031023  Alewife_Trophy         灰西鲱  Fish_Alewife_Trophy
3  4000040     300040  101031460  Alewife_Unique         灰西鲱  Fish_Alewife_Unique
4  4000050     300050  101031570    Alewife_Apex         灰西鲱    Fish_Alewife_Apex
